# M1 useful nodes
This notebook identifies M1 nodes (attention heads and MLPs) that, when ablated, cause a decrease in model prediction accuracy. These nodes are needed (aka useful) for accurate predictions.


This notebook was:
- Developed on Google Colab using an **T4**
- Runs with M1 (TinyStories) with base/CS1/CS2/CS3.
- Requires a GITHUB_TOKEN secret to access Martian quanta_text_to_sql code repository.
- Requires a HF_TOKEN secret to access Martian HuggingFace repository.

This notebook relies on the nnsight library. Useful background:
- https://nnsight.net/notebooks/tutorials/walkthrough/#Batching
- https://nnsight.net/notebooks/tutorials/walkthrough/#Looping

This notebook relies on the https://github.com/PhilipQuirke/quanta_mech_interp library for graphing useful nodes.

# Import libraries
Imports standard libraries. Do not read.

In [ ]:
# https://nnsight.net/
# Access 0.4 prerelease version (as at Dec 2024)
#!pip install nnsight==0.4.0.dev0
!pip install -U nnsight

In [ ]:
from IPython.display import clear_output
import einops
import torch
import numpy as np
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "colab"
import matplotlib.pyplot as plt
import tqdm.auto as tqdm

import nnsight
from nnsight import LanguageModel

In [ ]:
from getpass import getpass
from google.colab import userdata
import gc
import weakref

In [ ]:
!pip install --upgrade git+https://github.com/PhilipQuirke/quanta_mech_interp.git
import QuantaMechInterp as qmi

In [ ]:
github_token = userdata.get("GITHUB_TOKEN")

# Install the private repository using the token
!pip install --upgrade git+https://{github_token}@github.com/withmartian/quanta_text_to_sql.git

import QuantaTextToSql as qts

# Select model, command set and feature to investigate


In [ ]:
model_num = 1                 # 1=TinyStories, 2=Qwen, 3=Llama, 4=Granite, 5=SmolLM
cs_num = 1                    # 0=BaseModel, 1=CS1, 2=CS2 or 3=CS3

# Load model

In [ ]:
model = qts.load_tinysql_model(model_num, cs_num, auth_token=userdata.get("HF_TOKEN"))
clear_output()
print(model)

In [ ]:
N_LAYERS, N_HEADS, D_MODEL, D_HEAD = qts.get_model_sizes(model_num, model)

In [ ]:
model_hf_name = qts.sql_interp_model_location(model_num, cs_num)

# Singleton QuantaTool "main" configuration class. qmi.AlgoConfig is derived from the chain qmi.UsefulConfig > qmi.ModelConfig
cfg = qmi.AlgoConfig()
cfg.repo_name, cfg.model_name = model_hf_name.rsplit("/", 1)
cfg.main_model = model
cfg.n_layers = N_LAYERS
cfg.n_heads = N_HEADS
cfg.d_model = D_MODEL
cfg.d_head = D_HEAD
cfg.file_config_prefix = ""

print("cfg.repo_name="+cfg.repo_name, "cfg.model_name="+cfg.model_name)

# Generate test data and experiment runs

In [ ]:
# Generate a batch of prompts with 3 field names
def generate_batch(batch_size):
    cfg.batch_size = batch_size

    return qts.generate_csn(batch_size=batch_size, csn=max(1,cs_num), min_cols=3, max_cols=3)

In [ ]:
def same_string(title, token_idx, input_tokens, output_tokens):
  decoded_input = model.tokenizer.decode(input_tokens, skip_special_tokens=True)
  decoded_output = model.tokenizer.decode(output_tokens, skip_special_tokens=True)
  same = decoded_input==decoded_output
  if not same:
      print(title, ": token_idx="+str(token_idx), ": Same="+str(same))

In [ ]:
def run_attention_experiment(run_layer_idx, run_head_idx, run_token_idx, all_tokens, num_prompt_tokens, num_generate, make_changes = True):

    num_tokens = len(all_tokens)
    assert num_tokens == num_prompt_tokens + num_generate

    start = run_head_idx * D_HEAD
    end = (run_head_idx + 1) * D_HEAD

    if not make_changes:
         with model.generate(all_tokens[:num_prompt_tokens], max_new_tokens=num_generate,
                          pad_token_id=model.tokenizer.eos_token_id) as tracer:

            final_output = model.generator.output.save()

    elif run_token_idx < num_prompt_tokens:
        with model.generate(all_tokens[:num_prompt_tokens], max_new_tokens=num_generate,
                          pad_token_id=model.tokenizer.eos_token_id) as tracer:

            # Zero out just the portion of the prompt corresponding to this head
            model.transformer.h[run_layer_idx].output[0][:, run_token_idx, start:end] = 0

            final_output = model.generator.output.save()

    else:
        assert num_prompt_tokens == run_token_idx
        if False: # This doesnt seem to work
            with model.generate(all_tokens[:num_prompt_tokens], max_new_tokens=num_generate,
                              pad_token_id=model.tokenizer.eos_token_id) as tracer:

                for next_idx in range(num_generate):
                    model.transformer.h[run_layer_idx].next()

                # Zero out just the portion of the output corresponding to this head
                # PQR: Runs but output is never corrupted. This code run inside above for loop when next_idx == 0 crashes. Sigh
                model.transformer.h[run_layer_idx].output[0][:, run_token_idx, start:end] = 0

                final_output = model.generator.output.save()
        elif num_generate > 1:
            # Try this.
            with model.generate(all_tokens[:num_prompt_tokens+1], max_new_tokens=num_generate-1,
                              pad_token_id=model.tokenizer.eos_token_id) as tracer:

                # Zero out just the portion of the prompt corresponding to this head
                model.transformer.h[run_layer_idx].output[0][:, run_token_idx, start:end] = 0

                final_output = model.generator.output.save()
        else:
            return True, "", ""

    #final_output = final_output.detach().cpu().numpy()

    # Did the output change?
    decoded_input = model.tokenizer.decode(all_tokens, skip_special_tokens=True)
    decoded_output = model.tokenizer.decode(final_output[0], skip_special_tokens=True)
    same = decoded_input==decoded_output

    return same, decoded_input, decoded_output

In [ ]:
# Return a list of experiments to run
def get_experiment_list(examples, by_attention_head):
    run_list = []
    max_prompt_tokens = 0
    max_answer_tokens = 0
    max_tokens = 0
    show_one_example = True

    for example in examples:
        prompt = example.get_alpaca_prompt()
        answer = example.sql_statement

        prompt_tokens = model.tokenizer(prompt)["input_ids"]
        answer_tokens = model.tokenizer(answer)["input_ids"]
        all_tokens = prompt_tokens + answer_tokens

        num_answer_tokens = len(answer_tokens)
        num_prompt_tokens = len(prompt_tokens)
        num_tokens = len(all_tokens)
        assert num_tokens == num_prompt_tokens + num_answer_tokens

        max_prompt_tokens = max(max_prompt_tokens, num_prompt_tokens)
        max_answer_tokens = max(max_answer_tokens, num_answer_tokens)
        max_tokens = max(max_tokens, num_tokens)

        # Check that the model can predict the question
        same, _, decoded_output = run_attention_experiment(0, 0, 0, all_tokens, num_prompt_tokens, num_answer_tokens, make_changes = False)
        if not same:
            print( "Ignoring example that model doesn't predict correctly:", prompt.replace('\n', ' '), answer.replace('\n', ' ') )
            continue
        elif show_one_example:
            print("Example model predict correctly:", prompt.replace('\n', ' '), answer.replace('\n', ' ') )
            show_one_example = False;

        if by_attention_head:
            for layer_idx in range(N_LAYERS):
                for head_idx in range(N_HEADS):
                    for token_idx in range(num_tokens):
                        exp_prompt_tokens = max(num_prompt_tokens, token_idx)
                        exp_num_generate = num_tokens - exp_prompt_tokens
                        assert num_tokens == exp_prompt_tokens + exp_num_generate

                        run_list.append([layer_idx, head_idx, token_idx, all_tokens, exp_prompt_tokens, exp_num_generate])
        else:
            for token_idx in range(num_prompt_tokens):
                run_list.append([prompt, answer, token_idx, prompt_tokens])

    return run_list, max_prompt_tokens, max_answer_tokens, max_tokens

# Which token positions are useful?
This information is used to shrink the size of search spaces in following sections. For the SQL model all token positions are useful

In [ ]:
all_token_positions_useful = True

In [ ]:
N_BATCH = 50

def run_token_experiments( ):
    examples = generate_batch(N_BATCH)
    run_list, max_prompt_tokens, max_good_answer_tokens, max_tokens = get_experiment_list(examples, False)
    cfg.initialize_token_positions( max_prompt_tokens, max_good_answer_tokens, True )
    num_exps = len(run_list)

    print("N_BATCH="+str(N_BATCH), "max_prompt_tokens="+str(max_prompt_tokens), "max_good_answer_tokens="+str(max_good_answer_tokens), "max_tokens="+str(max_tokens), "num_exps="+str(num_exps))

    try_results = np.zeros(max_tokens, dtype=int)
    fail_results = np.zeros(max_tokens, dtype=int)

    if not all_token_positions_useful:
        for item_num in tqdm.tqdm(range(num_exps)):

            run_item = run_list[item_num]
            run_prompt, run_answer, run_token_idx, prompt_tokens = run_item

            with model.generate(prompt_tokens, max_new_tokens=max_good_answer_tokens,
                              pad_token_id=model.tokenizer.eos_token_id) as tracer:

                # Zero out just the portion of the output corresponding to this token position
                for run_layer_idx in range(N_LAYERS):
                    model.transformer.h[run_layer_idx].output[0][:, run_token_idx, :] = 0

                final_output = model.generator.output.save()

            final_output = final_output.detach().cpu().numpy()
            decoded_output = model.tokenizer.decode(final_output[0], skip_special_tokens=True)

            # Did the output change?
            try_results[run_token_idx] += 1
            if run_prompt + run_answer != decoded_output:
                #print("Input:", item_num, run_prompt.replace('\n', ' '), run_answer.replace('\n', ' '))
                #print("Output:", item_num, decoded_output.replace('\n', ' '))
                fail_results[run_token_idx] += 1

    # Compute the failure rate as percentage
    failure_rate = (1.0 * fail_results / (try_results + 1e-10)) * 100
    failure_rate = np.round(failure_rate, 2)

    return max_tokens, fail_results, failure_rate

g_max_tokens, g_token_fail_results, g_token_failure_rate = run_token_experiments()

In [ ]:
if all_token_positions_useful:
    for token_idx in range(g_max_tokens):
        cfg.add_useful_position(token_idx)
else:
    print( "Useful token positions:" )
    for token_idx in range(len(g_token_failure_rate)):
        if g_token_failure_rate[token_idx] > 0 :
            cfg.add_useful_position(token_idx)
            print( "Position:", token_idx, "% Fails:", g_token_failure_rate[token_idx], "# Fails:", g_token_fail_results[token_idx] )

In [ ]:
if False:
    cfg.calc_position_failures_map(g_token_fail_results.tolist())
    qmi.save_plt_to_file(cfg=cfg, full_title="Failures When Position Ablated")
    plt.show()

# Which token+layer+attention head nodes are useful?

In [ ]:
N_BATCH = 5

def run_attention_experiments():
    show_diff = True
    examples = generate_batch(N_BATCH)
    run_list, max_prompt_tokens, max_good_answer_tokens, max_tokens = get_experiment_list(examples, True)
    num_exps = len(run_list)

    print("N_BATCH="+str(N_BATCH), "max_prompt_tokens="+str(max_prompt_tokens), "max_answer_tokens="+str(max_good_answer_tokens), "max_tokens="+str(max_tokens), "num_exps="+str(num_exps))

    try_results = np.zeros((N_LAYERS, N_HEADS, max_tokens), dtype=int)
    fail_results = np.zeros((N_LAYERS, N_HEADS, max_tokens), dtype=int)

    for item_num in tqdm.tqdm(range(num_exps)):

        run_item = run_list[item_num]
        run_layer_idx, run_head_idx, run_token_idx, all_tokens, num_prompt_tokens, num_generate = run_item

        same, decoded_input, decoded_output = run_attention_experiment(run_layer_idx, run_head_idx, run_token_idx, all_tokens, num_prompt_tokens, num_generate, make_changes=False)
        if not same:
            print("Unexpected failure when no intervention", run_layer_idx, run_head_idx, run_token_idx, num_prompt_tokens, num_generate)
            print("Input :", decoded_input.replace('\n', ' '))
            print("Output:", decoded_output.replace('\n', ' '))
            assert False

        same, decoded_input, decoded_output = run_attention_experiment(run_layer_idx, run_head_idx, run_token_idx, all_tokens, num_prompt_tokens, num_generate)

        if not same:
            fail_results[run_layer_idx, run_head_idx, run_token_idx] += 1
            if show_diff:
                print("Failure when intervening", run_layer_idx, run_head_idx, run_token_idx, num_prompt_tokens, num_generate)
                print("Input :", decoded_input.replace('\n', ' '))
                print("Output:", decoded_output.replace('\n', ' '))
                show_diff = False
        try_results[run_layer_idx, run_head_idx, run_token_idx] += 1


    # Compute the failure rate as percentage
    failure_rate = (1.0 * fail_results / (try_results + 1e-10)) * 100
    failure_rate = np.round(failure_rate, 2)

    return max_tokens, fail_results, failure_rate, try_results

g_max_tokens, g_attn_failure_results, g_attn_failure_rate, g_attn_try_results = run_attention_experiments()

In [ ]:
cfg.useful_nodes = qmi.UsefulNodeList()
for layer_idx in range(N_LAYERS):
    for head_idx in range(N_HEADS):
        for token_idx in range(g_max_tokens):
            fail_perc = int(g_attn_failure_rate[layer_idx, head_idx, token_idx])
            if fail_perc > 0 :
                # Add percentage failure quanta
                node_location = qmi.NodeLocation(token_idx, layer_idx, True, head_idx)
                cfg.add_useful_node_tag( node_location, qmi.QType.FAIL.value, str(fail_perc) )

cfg.useful_nodes.sort_nodes()

In [ ]:
for layer_idx in range(N_LAYERS):
    plt.imshow(g_attn_failure_rate[layer_idx], cmap="viridis", aspect="auto")
    plt.colorbar(label="Percentage Change")
    plt.xlabel("Token Position")
    plt.ylabel("Attention Head")
    plt.title("Percentage of Output Changes by Zeroing Activations in Layer " + str(layer_idx))
    plt.show()

In [ ]:
cfg.useful_nodes.print_node_tags()

In [ ]:
title = "Useful attention heads"
ax1, quanta_results, num_results = qmi.calc_quanta_map(
    cfg, True, 6,
    cfg.useful_nodes, qmi.QType.FAIL.value, "", qmi.get_quanta_fail_perc,
    combine_identical_cells=False)

if num_results > 0:
    if cfg.graph_file_suffix > "":
        print("Saving quanta map:", title)
        qmi.save_plt_to_file(cfg=cfg, full_title=title)
    else:
        ax1.set_title(title + ' ({} nodes)'.format(len(quanta_results)))

    plt.show()

In [ ]:
# Serialize and save the useful nodes list to a temporary CoLab file in JSON format
main_fname_behavior_json = cfg.model_name + '_behavior.json'
print( "Saving useful node list with behavior tags:", main_fname_behavior_json)
cfg.useful_nodes.save_nodes(main_fname_behavior_json)